In [ ]:
!pip install idx2numpy
import numpy as np
import idx2numpy as id
import matplotlib.pyplot as plt
import math
 
dataset=id.convert_from_file('/content/drive/MyDrive/Untitled folder/train-images.idx3-ubyte')
data_label=id.convert_from_file('/content/drive/MyDrive/Untitled folder/train-labels.idx1-ubyte')
 
print(dataset.shape)
 
print(data_label)
#img=(dataset[0,:,:])
#plt.imshow(img)
 
y=np.zeros((10,60000))
for i in range(60000):
  y[data_label[i]][i]=1.0
x_data=dataset.reshape(60000,784).T
x=x_data/255
X_minibatch={}
Y_minibatch={}
for i in range(234):
  X_minibatch["X"+str(i+1)]=x[:,i*256:256*(i+1)]
  Y_minibatch["Y"+str(i+1)]=y[:,i*256:256*(i+1)]
print(x.shape)
print(y.shape, x[344,466])
 
def relu_base(z):
     if z<0:
       return 0 
     if z>=0:
        return z
relu=np.vectorize(relu_base)
 
 
 
def softmax(x):
    max=np.max(x,axis=0)
    ex = np.exp(x - max)
    sum=np.sum(ex,axis=0)
    return ex / sum
 
 
 
#low =-np.sqrt(6)/(np.sqrt(lay[i]+lay[i-1])), high = np.sqrt(6)/(np.sqrt(lay[i]+lay[i-1])
def initialize(lay): 
  np.random.seed(0)
  para={}
  for i in range(1,len(lay)):
    np.random.seed(0)
    para["w"+str(i)]=np.random.rand(lay[i],lay[i-1])*np.sqrt(1/(lay[i]*lay[i-1]))
    para["b"+str(i)]=np.zeros((lay[i],1))
  return para
 
 
 
def der(a):
  if a<=0:
    a=0
  else:
    a=1.0
  return a
der_relu=np.vectorize(der)
 
 
 
def forward_prop(a,w,b,activation):
    z=np.dot(w,a)+b
    if activation=="relu":
        return np.maximum(0,z),z
    if activation=="softmax":
        return softmax(z),z
 
 
def calculate_cost(AL, Y):
    
    m = Y.shape[1]
    cost = (-1 / m) * np.sum(np.multiply(Y, np.log(AL)) )
    #print("cost="+str(cost))
    return cost
 
"""m=y.shape[1]
  log_al=np.log(al)
  loss=np.multiply(log_al,y)
  
  cost=(-1/m)*(np.sum(loss))
  cost=np.squeeze(cost)
  return cost
"""
 
def back_prop(dz,a_nxt,w):
  m=dz.shape[1]
  dw=1/m*np.dot(dz,(a_nxt.T))
  db=1/m*np.sum(dz,axis = 1,keepdims= True)
  da_nxt=np.dot(w.T,dz)
  return dw,db,da_nxt
 
def back_activation(z,da):
  dz=der_relu(z)*da
  return dz
 
def update_para(alp,grad,para):
  n=len(para)//2
  for i in range(1,n+1):
    para["w"+str(i)]=para["w"+str(i)]-(alp*grad["dw"+str(i)])
    para["b"+str(i)]=para["b"+str(i)]-(alp*grad["db"+str(i)])
  return para
 
def L_forward_prop(x,para):
  L=len(para)//2
  cache=[(x,x)]
  a=x
  for i in range(1,L):
     a_nxt,z=forward_prop(a,para["w"+str(i)],para["b"+str(i)],"relu")
     a=a_nxt
     cache.append((z,a))
 
  al,zl=forward_prop(a,para["w"+str(L)],para["b"+str(L)],"softmax")
  cache.append((zl,al))
  return al,cache
 
def relu_backward(dA, cache):
   
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    
    # When z <= 0, you should set dz to 0 as well. 
    dZ[Z <= 0] = 0
    
    assert (dZ.shape == Z.shape)
    
    return dZ
 
def L_back_prop(dzl,cache,para):
  grad={}
  for i in reversed(range(1,(len(para)//2)+1)):
      if i==(len(para)//2):
         dz = dzl
      else:
        dz= relu_backward(da_nxt,cache[i][0])  
      grad["dw"+str(i)], grad["db"+str(i)], da_nxt= back_prop(dz,cache[i-1][1],para["w"+str(i)])
      
  return grad
 
def train_model(xmb,ymb,lay):
  para=initialize(lay)
  cof=[]
  itr=[]
  for i in range(100):
    for j in range(1,235):
      al,cache=L_forward_prop(xmb["X"+str(j)],para)
      cost=calculate_cost(al,ymb["Y"+str(j)])
      if j%200==0:
         print(i," ","cost",cost) 
         cof.append(cost)
         itr.append(i)
      dzl=al-ymb["Y"+str(j)]
      grad=L_back_prop(dzl,cache,para)
      para=update_para(0.01,grad,para)
    
  plt.plot(itr,cof)
  plt.ylabel("costs")
  plt.xlabel("iteratins")
  plt.title("learning rate=0.01")
  plt.show()
  return para
 
 
      
 
  
 
layers=[784,256,10]
 
parameters=train_model(X_minibatch,Y_minibatch,layers)


FileNotFoundError: ignored

In [ ]:

def p(x,y,para):
    m=x.shape[1]
    co=0
    al,c=L_forward_prop(x,para)
    
    pd=np.zeros((10,m))
    
    for i in range(m):
      n=np.argmax(al[:,i:i+1],axis=0)
      pd[n,i]=1.0
      if (np.array_equal(pd[:,i:i+1],y[:,i:i+1])):
        
        co+=1
  
    
    print("accuracy = ",(co/m)*100)


test_dataset=id.convert_from_file("/content/drive/MyDrive/Untitled folder/t10k-images.idx3-ubyte")
test_data_label=id.convert_from_file("/content/drive/MyDrive/Untitled folder/t10k-labels.idx1-ubyte")
y_test=np.zeros((10,10000))

for i in range(10000):
  y_test[test_data_label[i]][i]=1.0
x_test_data=test_dataset.reshape(10000,784).T
x_test=x_test_data/255 


p(x,y,parameters)
p(x_test,y_test,parameters)

In [ ]:
i=np.random.randint(1,60000)
img=x[:,i:i+1]
lab=y[:,i:i+1]
res,c=L_forward_prop(img,parameters)
print(res.shape)

plt.imshow(dataset[i,:,:])
print("looks like ",np.argmax(res))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')